In [11]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sklearn

from sklearn import decomposition
from sklearn import datasets
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize


import pandas as pd

# Last ned datasett

In [22]:
np.random.seed(5)

dataset_wine_url = 'http://mlr.cs.umass.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
wine = pd.read_csv(dataset_wine_url, sep=';')

Y = wine.quality
X = wine.drop('quality', axis=1)


Oversikt over attributtene

In [6]:
print(wine.describe())

       fixed acidity  volatile acidity  citric acid  residual sugar  \
count    1599.000000       1599.000000  1599.000000     1599.000000   
mean        8.319637          0.527821     0.270976        2.538806   
std         1.741096          0.179060     0.194801        1.409928   
min         4.600000          0.120000     0.000000        0.900000   
25%         7.100000          0.390000     0.090000        1.900000   
50%         7.900000          0.520000     0.260000        2.200000   
75%         9.200000          0.640000     0.420000        2.600000   
max        15.900000          1.580000     1.000000       15.500000   

         chlorides  free sulfur dioxide  total sulfur dioxide      density  \
count  1599.000000          1599.000000           1599.000000  1599.000000   
mean      0.087467            15.874922             46.467792     0.996747   
std       0.047065            10.460157             32.895324     0.001887   
min       0.012000             1.000000         

# Tren en klassifiseringsmodell på datasettet

In [23]:
from sklearn.neighbors import KNeighborsClassifier
np.random.seed(0)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, 
                                                    test_size=0.2, 
                                                    random_state=123, 
                                                    stratify=Y)


knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, Y_train) 
knn.score(X_test, Y_test)


0.48125000000000001

# Feature selection

En enkel måte å redusere dimensjonene til et problem er å droppe de minst viktige attributtene, kalt feature selection. Det er mange måter å gjøre det på. Èn måte er å fjerne alle attributter som har en varians lavere enn et gitt nivå, som i eksempelet under. 

In [24]:
sel = VarianceThreshold(threshold=0.2)
sel.fit(X)
X_feature_selection = sel.transform(X)
print("Hvilke attributter som var viktige nok til å få være med " + str(sel.get_support()))

Hvilke attributter som var viktige nok til å få være med [ True False False  True False  True  True False False False  True]


## Oppgave 0

Hva er fordelen med å redusere dimensjonene med PCA fremfor på denne måten?

# PCA

## Oppgave 1 

Gjør en PCA på datasettet ved hjelp av scikit (http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html). 

Reduserer dimensjonen til X slik at den kun har 3 dimensjoner og lagre resultatet i X_pca. 

In [25]:
pca = decomposition.PCA(n_components=3)
pca.fit(X)
X_pca = pca.transform(X)

# Oppgave 2

PCA-analysen i forrige oppgave skal ha gitt deg en X-matrise som nå kun består av tre features/attributter. Disse tre attributtene er ikke lik noen av de opprinnelige attributtene, men hver nye attributt er en unik kombinasjon av alle de opprinnelige attributtene. I denne oppgaven skal vi se på hvor viktig hver av egenvektorene er for å beskrive datasettet.  

Hent ut en oversikt over hvor stor andel hver egenvektor har for å forklare variansen i datasettet. 

In [26]:
pca.explained_variance_ratio_

array([ 0.94657698,  0.0483683 ,  0.00258917])

## Oppgave 3

PCA-analysen i forrige oppgave skal ha gitt deg en X-matrise som nå kun består av tre features/attributter. Disse tre attributtene er ikke lik noen av de opprinnelige attributtene, men hver nye attributt er en unik kombinasjon av alle de opprinnelige attributtene. I denne oppgaven skal vi se hvordan vi kan beskrive de nye attributenne ved hjelp av de opprinnelige. 

Hent ut komponentene som PCA-analysen består av.

In [27]:
pca.components_

array([[ -6.13247431e-03,   3.84465551e-04,   1.70902595e-04,
          8.64894648e-03,   6.37307290e-05,   2.18857434e-01,
          9.75678369e-01,   3.72498542e-06,  -2.68008619e-04,
          2.23381730e-04,  -6.35846721e-03],
       [ -2.38994985e-02,  -2.00966661e-03,  -3.03480788e-03,
          1.11348551e-02,  -2.36654751e-04,   9.75265982e-01,
         -2.18916841e-01,  -2.49998510e-05,   3.27182194e-03,
          6.18926046e-04,   1.45642451e-02],
       [  9.53135980e-01,  -2.51315387e-02,   7.37082746e-02,
          2.80913620e-01,   2.94578815e-03,   2.08968395e-02,
         -1.52685886e-03,   7.76139600e-04,  -5.86305467e-02,
          1.75252442e-02,  -4.85991164e-02]])

Du ser nå (forhåpentligvis) på en 3x4-matrise. Hver rad beskriver en egenvektor (en ny attributt) og innholder fire elementer som representerer de opprinnelige attributene. Hvert element sier deg i hvor stor grad hver egenvektoren beskrives av de opprinnelige attributten. 

# Plot resultatet

In [31]:
fig = plt.figure(1, figsize=(4, 3))
plt.clf()
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
plt.cla()

for name, label in [('3', 3), ('4', 4), ('5', 5),  ('6', 6),  ('7', 7),  ('8', 8)]:
    ax.text3D(X_pca[Y == label, 0].mean(),
              X_pca[Y == label, 1].mean() + 1.5,
              X_pca[Y == label, 2].mean(), name,
              horizontalalignment='center',
              bbox=dict(alpha=.5, edgecolor='w', facecolor='w'))

ax.scatter(X_pca[:, 0], X_pca[:, 1], X_pca[:, 2], c=Y, cmap=plt.cm.spectral,
           edgecolor='k')

ax.w_xaxis.set_ticklabels([])
ax.w_yaxis.set_ticklabels([])
ax.w_zaxis.set_ticklabels([])

[]

In [32]:
plt.show()

# Tren en klassifiseringsmodell på det dimensjonsreduserte datasettet

In [36]:
from sklearn.neighbors import KNeighborsClassifier
np.random.seed(0)
indices = np.random.permutation(len(X))
number_of_training_examples = int(0.8*len(X))

X_PCA_train=X_pca[indices[:number_of_training_examples]]
Y_train=Y[indices[:number_of_training_examples]]
X_PCA_test=X_pca[indices[number_of_training_examples:]]
Y_test=Y[indices[number_of_training_examples:]]

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_PCA_train, Y_train) 
knn.score(X_PCA_test, Y_test)

0.50312500000000004

## Oppgave 4

Forventet du et bedre eller dårligere resultat enn modellen du bygget tidligere på hele datasettet? Når ønsker du å bruke PCA?

# BONUS: Erstatt PCA med Cross Decomposition

Når vi reduserer dimensjonene med PCA ser vi kun på input-dataen (X), og finner de dimensjonene som beskriver datasettet best mulig slik at vi taper minst mulig informasjon når vi reduserer dimensjonene. 

En annen måte å redusere dimensjonene på er å ta i bruk det vi ønsker å predikere (Y) og velge de dimensjonene som beskriver Y best. 

In [ ]:
#Din kode her